In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from google.colab import drive

In [ ]:
import warnings
warnings.filterwarnings("ignore")


In [ ]:
file_path='train_data.parquet'

In [ ]:
df=pd.read_parquet(file_path,engine='auto')

FileNotFoundError: [Errno 2] No such file or directory: 'train_data.parquet'

In [ ]:
df.head()

In [ ]:
file_path_test='test_data.parquet'
test_df=pd.read_parquet(file_path_test,engine='auto')
train_cols = df.columns
test_cols = test_df.columns
iv_cols = [col for col in df.columns if "_iv_" in col]
# cols involving iv data
test_iv_cols = [col for col in test_df.columns if "_iv_" in col]
train_call_iv_cols = [col for col in train_cols if "call_iv_" in col]
train_put_iv_cols = [col for col in train_cols if "put_iv_" in col]
test_call_iv_cols = [col for col in test_cols if "call_iv_" in col]
test_put_iv_cols = [col for col in test_cols if "put_iv_" in col]
# cols involving iv data
# Columns in train but not in test, in original train order
train_spec_cols = [col for col in train_cols if col not in test_cols]
train_spec_call_iv_cols=[col for col in train_spec_cols if "call_iv_" in col]
train_spec_put_iv_cols=[col for col in train_spec_cols if "put_iv_" in col]
# Columns in test but not in train, in original test order
test_spec_cols = [col for col in test_cols if col not in train_cols]
test_spec_call_iv_cols=[col for col in test_spec_cols if "call_iv_" in col]
test_spec_put_iv_cols=[col for col in test_spec_cols if "put_iv_" in col]
common_call_iv_cols = [col for col in train_call_iv_cols if col in test_call_iv_cols]
common_put_iv_cols = [col for col in train_put_iv_cols if col in test_put_iv_cols]

In [ ]:
common_cols=common_call_iv_cols+common_put_iv_cols

In [ ]:
file_path='/content/drive/MyDrive/NK_Hack/df_ready_export.csv'
df_ready_2=pd.read_csv(file_path)
df_ready_2.drop(train_spec_cols,inplace=True,errors='ignore')
df_ready_2.shape

In [ ]:
file_path_test='/content/drive/MyDrive/NK_Hack/df_spline_fill.csv'
df_fire_imputed_spline=pd.read_csv(file_path_test)

In [ ]:
df.describe()

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_regression
import numpy as np



# Define parameter grids
param_grids = {
    "LinearRegression": {
        "regressor": [LinearRegression()]
    },
    "Ridge": {
        "regressor": [Ridge()],
        "regressor__alpha": [0.01, 0.1, 1.0, 10.0]
    },
    "MLP": {
        "regressor": [MLPRegressor(max_iter=500)],
        "regressor__hidden_layer_sizes": [(64,), (128, 64)],
        "regressor__alpha": [0.0001, 0.001],
        "regressor__learning_rate_init": [0.001, 0.01]
    },
    "XGB": {
        "regressor": [XGBRegressor(verbosity=0)],
        "regressor__n_estimators": [100, 200],
        "regressor__max_depth": [3, 5],
        "regressor__learning_rate": [0.05, 0.1]
    }
}

# Now you can use best_model like this:
# best_model.fit(X, y)  # Already fit above, but reuse on new data if needed

In [ ]:
df.shape

In [ ]:
file_path_test='test_data.parquet'
test_df=pd.read_parquet(file_path_test,engine='auto')
train_cols = df.columns
test_cols = test_df.columns
# Columns in train but not in test, in original train order
train_spec_cols = [col for col in train_cols if col not in test_cols]

# Columns in test but not in train, in original test order
test_spec_cols = [col for col in test_cols if col not in train_cols]

print("Ordered columns in train_data.parquet that are absent in test parquet:", train_spec_cols)
print("Ordered columns in test_data.parquet that are absent in train parquet:", test_spec_cols)


In [ ]:
file_path='df_ready_export.csv'
df_ready_2=pd.read_csv(file_path)
df_ready_2.drop(train_spec_cols,inplace=True,errors='ignore')
df_ready_2.shape

In [ ]:
file_path_test='df_spline_fill.csv'
df_fire_imputed_spline=pd.read_csv(file_path_test)

In [ ]:
df.drop(train_spec_cols,axis=1,inplace=True)

In [ ]:
df.shape

In [ ]:
iv_cols = [col for col in df.columns if "_iv_" in col]
# cols involving iv data

In [ ]:
test_iv_cols = [col for col in test_df.columns if "_iv_" in col]
# cols involving iv data

In [ ]:
#plot iv_cols from df_1 as histogram

df[iv_cols].hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
# prompt: cells with _iv_ columns should lie between 0 and 2

# Identify columns that contain '_iv_'
iv_cols = [col for col in df.columns if "_iv_" in col]

# Filter the DataFrame to keep only rows where all _iv_ columns are between 0 and 2
df_filtered = df[(df[iv_cols] >= 0.05).all(axis=1) & (df[iv_cols] <= 0.7).all(axis=1)]

# You can now work with df_filtered which contains the data that satisfies the condition
# print the shape of the filtered dataframe to see how many rows remain
print(f"Original shape: {df.shape}")
print(f"Filtered shape: {df_filtered.shape}")

# Optionally, you can replace the original dataframe with the filtered one
# df = df_filtered

In [ ]:
df_filtered[iv_cols].hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
scaler = StandardScaler()

In [ ]:
def preprocess(ds_frame):
  drop_worthy=['timestamp']
  filtered_df=ds_frame.drop(drop_worthy,axis=1)
  cols_to_scale = [col for col in filtered_df.columns if '_iv_' not in col]

# Apply StandardScaler to the identified columns
  filtered_df[cols_to_scale] = scaler.fit_transform(filtered_df[cols_to_scale])
  return filtered_df

In [ ]:
df_ready=preprocess(df_filtered)
df_ready.head()

In [ ]:
df_fire=preprocess(test_df)
df_fire.head()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error, r2_score, mean_absolute_error

In [ ]:
lr=LinearRegression()

In [ ]:
df_fire[test_spec_cols]

In [ ]:
from sklearn.impute import SimpleImputer,KNNImputer
simp_imp=SimpleImputer(missing_values=np.nan, strategy='mean')
knn_imputer=KNNImputer(n_neighbors=5)

In [ ]:
df_fire_copy=df_fire.copy()
df_fire_copy.drop('underlying',axis=1,inplace=True)
from sklearn.impute import KNNImputer
import pandas as pd



# Transpose to treat rows as features (so KNN will work across columns)
df_fire_copy_T = df_fire_copy.T

# KNN Imputer (now imputes across original columns)
knn_imputer = KNNImputer(n_neighbors=3)
df_fire_copy_imputed_T = pd.DataFrame(
    knn_imputer.fit_transform(df_fire_copy_T),
    index=df_fire_copy_T.index,
    columns=df_fire_copy_T.columns
)
# Transpose back to original shape — this was the bug
df_fire_copy = df_fire_copy_imputed_T.T

# Add back 'underlying' column (assumed to be untouched)
df_fire_copy = pd.concat([df_fire['underlying'], df_fire_copy], axis=1)

# Fallback mean imputation for any remaining NaNs
simp_imp = SimpleImputer(missing_values=np.nan, strategy='mean')
df_fire_imputed_copy = simp_imp.fit_transform(df_fire_copy)

# Convert back to DataFrame
df_fire_imputed_copy = pd.DataFrame(df_fire_imputed_copy, columns=df_fire_copy.columns)

# Preview
df_fire_imputed_copy.head()

In [ ]:
df_test_spec_imputed_cols=df_fire_imputed_copy[test_spec_cols]
df_test_spec_imputed_cols.head()

In [ ]:
df_attack=df_fire.drop(test_spec_cols,axis=1)
df_attack.head()

In [ ]:
df_fire_imputed_copy_trainer=df_fire_imputed_copy.drop(test_spec_cols,axis=1)
df_fire_imputed_copy_trainer.head()

In [ ]:
# prompt: plot of first row  test_iv_cols in df fire, colour predicted values in differnt colour

import matplotlib.pyplot as plt
# Assuming df_fire and submission_2_3 are already defined and contain the necessary data
# Get the first row of df_fire for original values
original_first_row = df_fire.iloc[754]

# Get the first row of submission_2_3 for predicted values
predicted_first_row = df_fire_imputed_copy.iloc[754]

# Identify the _iv_ columns from the original dataframe (or test_iv_cols list)
iv_cols_to_plot = test_iv_cols # Using the existing list from the preceding code

# Extract original and predicted IV values for the first row
original_iv_values = original_first_row[iv_cols_to_plot]
predicted_iv_values = predicted_first_row[iv_cols_to_plot]

# Create the plot
plt.figure(figsize=(14, 7))

# Plot original values
plt.scatter(iv_cols_to_plot, original_iv_values, color='blue', label='Original Values', marker='o')

# Plot predicted values in a different color
plt.scatter(iv_cols_to_plot, predicted_iv_values, color='red', label='Predicted Values', marker='x')

# Add labels and title
plt.xlabel('IV Columns')
plt.ylabel('IV Value')
plt.title('Original vs. Predicted IV Values for the First Sample in Test Data')
plt.xticks(rotation=90)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
df_trial=df_attack.copy()

In [ ]:
iv_cols

In [ ]:
df_fire_3=df_fire.copy()
df_fire_3.head()

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler


knn_reg = KNeighborsRegressor(n_neighbors=5)



In [ ]:
for iv in common_call_iv_cols:
  X=df_ready.drop([iv] + common_put_iv_cols, axis=1)
  y=df_ready[iv]

  # Track best model
  best_score = -np.inf
  best_model = None
  best_model_name = None

  # Grid search
  for name, params in param_grids.items():
      pipe = Pipeline([,
          ("regressor", params["regressor"][0])
      ])
      grid_search = GridSearchCV(pipe, param_grid=params, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
      grid_search.fit(X, y)

      print(f"{name} best score: {-grid_search.best_score_:.4f}")

      if grid_search.best_score_ > best_score:
          best_score = grid_search.best_score_
          best_model = grid_search.best_estimator_
          best_model_name = name
      best_model.fit(X, y)
  print(f"\nBest model: {best_model_name} with MSE: {-best_score:.4f}")

  df_fire_imputed_copy_trainer_predictor=df_fire_imputed_copy_trainer.drop([iv] + common_put_iv_cols,axis=1)
  for i in range(len(df_attack)):
    if np.isnan(df_fire.loc[i, iv]):
      df_fire_3.loc[i, iv] = best_model.predict(df_fire_imputed_copy_trainer_predictor.loc[i].values.reshape(1, -1))

In [ ]:
for iv in common_put_iv_cols:
  X=df_ready.drop([iv] + common_call_iv_cols, axis=1)
  y=df_ready[iv]

  # Track best model
  best_score = -np.inf
  best_model = None
  best_model_name = None

  # Grid search
  for name, params in param_grids.items():
      pipe = Pipeline([,
          ("regressor", params["regressor"][0])
      ])
      grid_search = GridSearchCV(pipe, param_grid=params, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
      grid_search.fit(X, y)

      print(f"{name} best score: {-grid_search.best_score_:.4f}")

      if grid_search.best_score_ > best_score:
          best_score = grid_search.best_score_
          best_model = grid_search.best_estimator_
          best_model_name = name
  best_model.fit(X, y)
  df_fire_imputed_copy_trainer_predictor=df_fire_imputed_copy_trainer.drop([iv] + common_call_iv_cols,axis=1)
  for i in range(len(df_attack)):
    if np.isnan(df_fire.loc[i, iv]):
      df_fire_3.loc[i, iv] = best_model.predict(df_fire_imputed_copy_trainer_predictor.loc[i].values.reshape(1, -1))


In [ ]:

df_fire_3.to_csv('trial.csv',index=False)

In [ ]:
df_fire_3[test_spec_cols]

In [ ]:
df_fire_3[common_cols].describe().loc['max']

In [ ]:
from sklearn.impute import KNNImputer
import pandas as pd



# Transpose to treat rows as features (so KNN will work across columns)
df_trial_T = df_trial.T

# KNN Imputer (now imputes across original columns)
knn_imputer = KNNImputer(n_neighbors=3)
df_fire_copy_imputed_T = pd.DataFrame(
    knn_imputer.fit_transform(df_fire_copy_T),
    index=df_fire_copy_T.index,
    columns=df_fire_copy_T.columns
)
# Transpose back to original shape — this was the bug
df_fire_copy = df_fire_copy_imputed_T.T

# Add back 'underlying' column (assumed to be untouched)
df_fire_copy = pd.concat([df_fire['underlying'], df_fire_copy], axis=1)

# Fallback mean imputation for any remaining NaNs
simp_imp = SimpleImputer(missing_values=np.nan, strategy='mean')
df_fire_imputed_copy = simp_imp.fit_transform(df_fire_copy)

# Convert back to DataFrame
df_fire_imputed_copy = pd.DataFrame(df_fire_imputed_copy, columns=df_fire_copy.columns)

# Preview
df_fire_imputed_copy.head()

In [ ]:
jdf=pd.read_csv('trial.csv')
jdf.head()

In [ ]:
# prompt: jdf cells with _iv_ columns should be below test_df[iv_cols].describe().loc['max'].max()

#jdf[iv_cols] = jdf[iv_cols].where( (jdf[iv_cols] < test_df[iv_cols].describe().loc['max'].max() ), np.nan)
#jdf[iv_cols] = jdf[iv_cols].fillna(method='ffill').fillna(method='bfill')

In [ ]:
jdf[iv_cols].describe().loc['max']

In [ ]:
jdf.columns

In [ ]:
test_cols

In [ ]:
jdf_call_iv_cols=[col for col in jdf.columns if "call_iv_" in col]
jdf_call_iv_cols

In [ ]:
jdf_put_iv_cols=[col for col in jdf.columns if "put_iv_" in col]
jdf_put_iv_cols

In [ ]:
test_spec_call_iv_cols=[col for col in test_spec_cols if "call_iv_" in col]
test_spec_call_iv_cols

In [ ]:
test_spec_put_iv_cols=[col for col in test_spec_cols if "put_iv_" in col]
test_spec_put_iv_cols

In [ ]:
submission_2_1=pd.concat([test_df['timestamp'],jdf[jdf_call_iv_cols],df_test_spec_imputed_cols[test_spec_call_iv_cols],jdf[jdf_put_iv_cols],df_test_spec_imputed_cols[test_spec_put_iv_cols]],axis=1)
submission_2_1.head()

In [ ]:
df_fire_2=df_fire.copy()
df_fire_2[df_trial.columns]=df_trial[df_trial.columns]
df_fire_2.head()

In [ ]:
submission_2_1.to_csv('submission_2_2.csv',index=False)

In [ ]:
submission_2_1.describe().loc['max']

In [ ]:
submission_2_1[iv_cols].describe().loc['max'].max()

In [ ]:
test_df[iv_cols].describe().loc['max'].max()

In [ ]:
for iv in test_spec_call_iv_cols:
  X=df_fire_imputed_copy.drop([iv]+test_put_iv_cols, axis=1)
  y=df_fire_imputed_copy[iv]

  # Track best model
  best_score = -np.inf
  best_model = None
  best_model_name = None

  # Grid search
  for name, params in param_grids.items():
      pipe = Pipeline([,
          ("regressor", params["regressor"][0])
      ])
      grid_search = GridSearchCV(pipe, param_grid=params, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
      grid_search.fit(X, y)

      print(f"{name} best score: {-grid_search.best_score_:.4f}")

      if grid_search.best_score_ > best_score:
          best_score = grid_search.best_score_
          best_model = grid_search.best_estimator_
          best_model_name = name
  best_model.fit(X, y)
  df_fire_imputed_copy_trainer_predictor_2=df_fire_imputed_copy.drop([iv]+test_put_iv_cols,axis=1)
  for i in range(len(df_attack)):
    if np.isnan(df_fire_2.loc[i, iv]):
      df_fire_3.loc[i, iv] = best_model.predict(df_fire_imputed_copy_trainer_predictor_2.loc[i].values.reshape(1, -1))


In [ ]:
for iv in test_spec_put_iv_cols:
  X=df_fire_imputed_copy.drop([iv]+test_call_iv_cols, axis=1)
  y=df_fire_imputed_copy[iv]

  # Track best model
  best_score = -np.inf
  best_model = None
  best_model_name = None

  # Grid search
  for name, params in param_grids.items():
      pipe = Pipeline([,
          ("regressor", params["regressor"][0])
      ])
      grid_search = GridSearchCV(pipe, param_grid=params, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
      grid_search.fit(X, y)

      print(f"{name} best score: {-grid_search.best_score_:.4f}")

      if grid_search.best_score_ > best_score:
          best_score = grid_search.best_score_
          best_model = grid_search.best_estimator_
          best_model_name = name
  best_model.fit(X, y)
  df_fire_imputed_copy_trainer_predictor_2=df_fire_imputed_copy.drop([iv]+test_call_iv_cols, axis=1)
  for i in range(len(df_attack)):
    if np.isnan(df_fire_2.loc[i, iv]):
      df_fire_3.loc[i, iv] = best_model.predict(df_fire_imputed_copy_trainer_predictor_2.loc[i].values.reshape(1, -1))

df_fire_3.to_csv('trial_2.csv',index=False)

In [ ]:
df_fire_3.to_csv('trial_2.csv',index=False)

In [ ]:
df_fire_2.head()

In [ ]:
submission_2_3=pd.concat([test_df['timestamp'],df_fire_3[test_iv_cols]],axis=1)
submission_2_3.head()

In [ ]:
#submission_2_3[test_iv_cols] = submission_2_3[test_iv_cols].where( (submission_2_3[test_iv_cols] < test_df[test_iv_cols].describe().loc['max'].max() ), np.nan)
#submission_2_3[test_iv_cols] = submission_2_3[test_iv_cols].fillna(method='ffill').fillna(method='bfill')

In [ ]:
submission_2_3[test_iv_cols].describe().loc['max'].max()

In [ ]:
submission_2_3.shape

In [ ]:
submission_2_3.to_csv('submission_2_6.csv',index=False)

In [ ]:
submission_2_3.head()

In [ ]:
submission_2_3.isna().sum().sort_values(ascending=False)

In [ ]:
# prompt: #scatter PLOT first sample of OF SUBMISSION 2_3,for the columns having_iv on their name

import matplotlib.pyplot as plt
# Get the first sample (row) of submission_2_3
first_sample = submission_2_3.iloc[0]

# Identify columns in the first sample that have "_iv_" in their name
iv_cols_sample = [col for col in first_sample.index if "_iv_" in col]

# Extract the values for the identified columns
iv_values_sample = first_sample[iv_cols_sample]

# Create a scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(iv_cols_sample, iv_values_sample, marker='o')
plt.xticks(rotation=90)
plt.ylabel('IV Value')
plt.title('Scatter Plot of IV Values for the First Sample in Submission_2_3')
plt.tight_layout()
plt.show()

In [ ]:
# prompt: #scatter PLOT first sample of OF SUBMISSION 2_3,for the columns having_iv on their name

import matplotlib.pyplot as plt
# Get the first sample (row) of submission_2_3
first_sample = df_fire_imputed_copy.iloc[754]

# Identify columns in the first sample that have "_iv_" in their name
iv_cols_sample = [col for col in first_sample.index if "_iv_" in col]

# Extract the values for the identified columns
iv_values_sample = first_sample[iv_cols_sample]

# Create a scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(iv_cols_sample, iv_values_sample, marker='o')
plt.xticks(rotation=90)
plt.ylabel('IV Value')
plt.title('Scatter Plot of IV Values for the First Sample in Submission_2_3')
plt.tight_layout()
plt.show()

In [ ]:
# prompt: plot of first row  test_iv_cols in df fire, colour predicted values in differnt colour

import matplotlib.pyplot as plt
# Assuming df_fire and submission_2_3 are already defined and contain the necessary data
# Get the first row of df_fire for original values
original_first_row = df_fire.iloc[0]

# Get the first row of submission_2_3 for predicted values
predicted_first_row = submission_2_3.iloc[0]

# Identify the _iv_ columns from the original dataframe (or test_iv_cols list)
iv_cols_to_plot = test_iv_cols # Using the existing list from the preceding code

# Extract original and predicted IV values for the first row
original_iv_values = original_first_row[iv_cols_to_plot]
predicted_iv_values = predicted_first_row[iv_cols_to_plot]

# Create the plot
plt.figure(figsize=(14, 7))

# Plot original values
plt.scatter(iv_cols_to_plot, original_iv_values, color='blue', label='Original Values', marker='o')

# Plot predicted values in a different color
plt.scatter(iv_cols_to_plot, predicted_iv_values, color='red', label='Predicted Values', marker='x')

# Add labels and title
plt.xlabel('IV Columns')
plt.ylabel('IV Value')
plt.title('Original vs. Predicted IV Values for the First Sample in Test Data')
plt.xticks(rotation=90)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
itftj

In [ ]:
mean_absolute_error_test=[]
mean_absolute_error_train=[]
y_pred_test_list=[]
y_pred_train_list=[]
Y_test_list=[]
Y_train_list=[]

In [ ]:
for iv in iv_cols:
  X=filtered_df.drop(iv, axis=1)
  y=filtered_df[iv]
  X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.2, random_state=42)
  lr.fit(X_train, y_train)
  y_pred = lr.predict(X_train)
  mean_absolute_error_train.append(mean_absolute_error(y_train, y_pred))
  y_pred_test = lr.predict(X_validate)
  mean_absolute_error_test.append(mean_absolute_error(y_validate, y_pred_test))
  y_pred_test_list.append(y_pred_test)
  y_pred_train_list.append(y_pred)
  Y_test_list.append(y_validate)
  Y_train_list.append(y_train)

In [ ]:
#plot
plt.figure(figsize=(10, 6))
plt.plot(iv_cols[6:], mean_absolute_error_train[6:], marker='o', label='Train')
plt.plot(iv_cols[6:], mean_absolute_error_test[6:], marker='o', label='Test')

In [ ]:
y_test_df=pd.DataFrame(Y_test_list)
y_train_df=pd.DataFrame(Y_train_list)
y_pred_test_df=pd.DataFrame(y_pred_test_list)
y_pred_train_df=pd.DataFrame(y_pred_train_list)

In [ ]:
y_test_df.head()

In [ ]:
#plot y_pred_test vs y_test
plt.figure(figsize=(10, 6))
plt.scatter(y_validate, y_pred_test, label='Test Data')

In [ ]:
y_pred

In [ ]:
df_test_para=pd.read_parquet('test_data.parquet',engine='auto')

In [ ]:
timestamp_series=df_test_para['timestamp']

In [ ]:
df_test_para.drop("timestamp",axis=1,inplace=True)

In [ ]:
iv_cols_test=[col_test for col_test in df_test_para.columns if "_iv_" in col_test]
iv_cols_test

In [ ]:
# prompt: show the column names same in test_para and filtered_df

test_para_set = df_test_para.columns
filtered_df_cols = filtered_df.columns
print("Columns present in both dataframes:")
common_col_list=[]
for common_col in test_para_set.intersection(filtered_df_cols):
  common_col_list.append(common_col)

In [ ]:
df_test_p1=df_test_para[common_col_list]
df_test_p1.shape

In [ ]:
df_test_p1

In [ ]:
df_test_para_copy=df_test_para.copy()

In [ ]:
from sklearn.impute import SimpleImputer
simp_imp=SimpleImputer(missing_values=np.nan, strategy='median')
#use imputer
imputed_df_test_para_copy=simp_imp.fit_transform(df_test_para_copy)
imputed_df_test_para_copy=pd.DataFrame(imputed_df_test_para_copy,columns=df_test_para_copy.columns)
imputed_df_test_para_copy.head()

In [ ]:
df_target=imputed_df_test_para_copy[iv_cols_test]
df_target.head()

In [ ]:
#concatenate series_timestamp and  df_target
df_target=pd.concat([timestamp_series,df_target],axis=1)
df_target.head()

In [ ]:
df_target.to_csv('submission_1_1.csv',index=False)